# Global Societal Endangerment Index (GSEI)

Development notebook for step 6: Weighting and Aggregation

In [58]:
import pandas as pd
import numpy as np
import categories as my_cat
from sklearn.preprocessing import StandardScaler

In [59]:
# load data
df = pd.read_csv("data/processing/normalised_all_data.csv")
countries = pd.read_csv("data/processing/selected_countries.csv")
categories = my_cat.load('4.1')
categories.pop('pca')

df

,Temp Change max 2012-2022,Disaster Affected Population 2020-2025,Disaster Damage US$T 2020-2025,Undernourishment (%),Renewable Freshwater per Capita (m3),Population below Poverty Line (%),Health Expenditure per Capita (US$),RoW Health Expenditure (%),No Education (%),Education Expenditure (US$M),...,pol_PC4,pol_PC5,pol_PC6,water_air_PC1,water_air_PC2,water_air_PC3,disease_PC1,disease_PC2,disease_PC3,disease_PC4
0,0.201103,1.118868,-1.070589,1.766991,0.435339,0.000000,-0.242164,2.913045,2.561201,0.749734,...,-0.157403,0.451415,-0.505144,1.653817,-0.551784,0.344034,0.090144,-0.350564,-0.636542,-0.235712
1,-0.252046,0.875721,-1.070589,1.440552,-0.199196,0.000000,-0.352886,2.729465,0.507129,0.047849,...,-0.231028,-0.006220,-0.390847,0.824321,-0.093015,-0.409678,1.992797,-0.404285,0.069539,-0.540231
2,0.228989,-1.639780,-1.070589,-0.416401,-0.594059,-0.016378,-1.064797,-0.511609,-0.724402,1.046331,...,0.586811,-0.234792,0.104068,-0.578045,-0.176162,0.080052,-1.054480,-0.055411,0.311029,0.261080
3,2.346589,0.026095,1.183035,-0.944423,-0.168613,-1.068933,1.266343,-0.396103,-0.737235,1.888762,...,0.219355,-0.118686,0.035244,-1.481279,0.295753,-0.300041,-2.077185,0.333092,-0.497454,-0.189044
4,-0.344419,-1.314753,-1.070589,-0.913244,2.530224,-1.068933,1.385419,-0.396103,-0.340386,-1.144288,...,0.846905,0.356944,-0.048044,-1.443530,0.430280,-0.053021,-0.416271,-0.269504,1.131575,0.000319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.795825,0.377215,0.906685,-0.226457,-1.153487,-0.016378,-0.780593,-0.511609,-0.466226,-0.577439,...,-0.416857,0.316650,0.536857,0.145085,-0.335460,0.634324,0.605262,-0.551248,1.068391,-0.013318
188,-0.192701,0.672441,0.568280,2.085959,1.920806,0.000000,0.134847,2.729465,2.508131,0.047849,...,-0.461834,0.256579,-0.162456,0.529793,-0.633598,0.142639,0.921834,-0.635536,-0.785883,0.997726
189,-0.149216,1.105546,1.061886,0.214688,0.671789,0.000000,-1.226685,2.729465,-0.583080,-0.905192,...,-0.322633,-0.116275,0.317120,0.477926,-0.255339,-0.603978,3.092266,4.768779,2.565955,-0.075491
190,-0.778396,1.105492,-1.070589,1.952186,-0.176332,1.933058,-0.210247,-0.511609,-0.087556,0.662586,...,-0.524287,-0.338142,-0.182993,1.585180,-0.578395,-0.472125,2.681855,1.528531,-0.796329,0.600526


## Weighting & Aggregation


In [60]:
# sub-indicator weights (different approaches)
w_total = {
    'equal': {
        "env": 1,
        "pol": 1,
        "soc": 1,
        "eco": 1,
        "threat": 1
    },
    'focus1': {
        "env": 0.3,
        "pol": 0.2,
        "soc": 0.15,
        "eco": 0.2,
        "threat": 0.15
    },
    'focus2': {
        "env": 0.12,
        "pol": 0.28,
        "soc": 0.15,
        "eco": 0.23,
        "threat": 0.22
    },
    'focus3': {
        "env": 0.20,
        "pol": 0.25,
        "soc": 0.15,
        "eco": 0.18,
        "threat": 0.22
    },
}

# weights per sub-indicator
w_cat = {
    'env': {
    "Temp Change max 2012-2022": 1.5,
    "Disaster Affected Population 2020-2025": 1.5,
    "Disaster Damage US$T 2020-2025": 1,
    "Undernourishment (%)": 1,
    "Renewable Freshwater per Capita (m3)": 1,
    "water_air_PC1": 1,
    "water_air_PC2": 1,
    "water_air_PC3": 1
    
}, 'pol': {
    "pol_PC1": 1,
    "pol_PC2": 1,
    "pol_PC3": 1,
    "pol_PC4": 1,
    "pol_PC5": 1,
    "pol_PC6": 1
    
}, 'soc': {
    "Population below Poverty Line (%)": 1.2,
    "Health Expenditure per Capita (US$)": 1,
    "RoW Health Expenditure (%)": 1,
    "No Education (%)": 1,
    "Education Expenditure (US$M)": 1,
    "Crime Rate": 1
    
}, 'eco': {
    "Government Debt (% of GDP)": 1,
    "Total Reserves (% of External Debt)": 1,
    "Gini Coefficient": 1,
    "Inflation (%)": 1,
    "Youth Unemployment (%)": 1,
    "Renewable Energy Consumption (%)": 1,
    "Secure Internet Servers": 1.2
    
}, 'threat': {
    "Future Conflict Risk": 1.5,
    "Current Conflict Intensity": 2,
    "COVID-19 Expenditure per Capita (US$)": 1.5,
    "disease_PC1": 1,
    "disease_PC2": 1,
    "disease_PC3": 1,
    "disease_PC4": 1
}}

In [61]:
# build index with weights
def aggregate_subindices(df, w):
    dfw = pd.DataFrame(index=df.index)
    
    for cat in categories:
        dfw[cat] = 0
        sum_weights = sum(w[cat].values())
        
        for indicator, weight in w[cat].items():
            dfw[cat] += df[indicator] * (weight / sum_weights)
            
    return dfw

def aggregate_index(df, w, name):
    sum_weights = sum(w.values())
    x = pd.DataFrame(columns=[name], index=df.index)
    x[name] = 0
    
    for cat, weight in w.items():
        x[name] += df[cat] * (weight / sum_weights)
    
    return x  

In [62]:
sub = aggregate_subindices(df, w_cat)

scaler = StandardScaler()
subn = pd.DataFrame(scaler.fit_transform(sub), columns=sub.columns)

pd.concat([countries, subn], axis=1).sort_values(by='env', ascending=False).reset_index(drop=True)

,Country,env,pol,soc,eco,threat
0,Chad,3.509448,1.422429,1.784576,0.954580,1.717466
1,Somalia,3.436625,1.921386,1.476596,1.153100,1.960357
2,South Sudan,2.478010,1.787046,1.667051,0.532186,1.461830
3,Nigeria,2.355663,1.133869,1.766614,-0.093653,2.045370
4,Niger,2.351419,1.048928,2.900702,-0.815618,1.694166
...,...,...,...,...,...,...
187,Liechtenstein,-1.788572,-1.513955,-1.824345,-1.417027,-1.156841
188,Montenegro,-1.837272,-0.402425,0.204480,0.409680,-0.705535
189,Bhutan,-1.869393,-0.341098,-0.721134,-1.001312,-0.877635
190,Equatorial Guinea,-2.176203,1.122083,-0.510699,0.432200,0.163422


In [63]:
x0 = aggregate_index(subn, w_total['equal'], 'equal')
x1 = aggregate_index(subn, w_total['focus1'], 'focus1')
x2 = aggregate_index(subn, w_total['focus2'], 'focus2')
x3 = aggregate_index(subn, w_total['focus3'], 'focus3')

pd.concat([countries, x0, x1, x2, x3], axis=1).sort_values(by='focus3', ascending=False).reset_index(drop=True)

,Country,equal,focus1,focus2,focus3
0,Somalia,1.989613,2.161427,1.868364,2.027997
1,Yemen,1.993296,1.955626,2.025347,1.967312
2,Syrian Arab Republic,1.862281,1.895253,1.914125,1.896381
3,Chad,1.877700,2.053543,1.684496,1.874850
4,Sudan,1.779799,1.684591,1.831357,1.773021
...,...,...,...,...,...
187,New Zealand,-1.286886,-1.317847,-1.307788,-1.320240
188,Finland,-1.382494,-1.388734,-1.369869,-1.365140
189,Ireland,-1.486976,-1.630752,-1.415842,-1.504614
190,Denmark,-1.530330,-1.538372,-1.554399,-1.509382


In [64]:
final_index = pd.concat([x3, subn], axis=1)
final_index.columns = ['GSEI'] + list(sub.columns)

scaler = StandardScaler()
final_index['GSEI'] = scaler.fit_transform(final_index[['GSEI']])

final_index

,GSEI,env,pol,soc,eco,threat
0,2.131364,1.359784,1.953472,2.473828,1.438660,1.379587
1,1.008055,0.426008,0.486660,1.050863,1.743148,0.558510
2,-0.629525,-1.453353,-0.290722,-0.220462,0.214211,-0.647465
3,-0.631942,0.639489,-1.228405,-0.228575,-0.750666,-0.698256
4,-0.528644,-0.897611,0.829798,-0.353304,-1.318832,-0.716695
...,...,...,...,...,...,...
187,-0.705034,-0.196080,-0.955540,-1.451833,0.058908,-0.341101
188,2.475435,1.591218,1.831934,2.059264,2.607494,1.876573
189,1.365750,0.895471,-0.343965,0.440655,1.893063,2.661157
190,0.500720,0.516306,-0.131904,0.717885,0.333533,0.726979


## Modifications and Extensions

Endangerment groups: Very Low, Low, Medium, High, Very High
- sum and average index values

Shift the index values
- subtract minimum (of all index columns) to only have positive numbers
- multiply by a multiple of 10 so that the maximum is still below 100

In [81]:
all_min = np.floor(final_index.min().min() * 10) / 10.0

shifted_index = (final_index - all_min) * 10

pd.concat([countries, shifted_index], axis=1).sort_values(by='GSEI', ascending=False).reset_index(drop=True)

,Country,GSEI,env,pol,soc,eco,threat
0,Somalia,51.517948,60.366245,45.213858,40.765955,37.531000,45.603573
1,Yemen,50.754351,41.912176,44.319345,46.592637,52.074941,44.765726
2,Syrian Arab Republic,49.861840,44.922247,48.793597,40.581642,46.148216,42.668336
3,Chad,49.590925,61.094481,40.224293,43.845758,35.545804,43.174658
4,Sudan,48.309623,37.038110,44.764719,46.956910,44.069365,46.160837
...,...,...,...,...,...,...,...
187,New Zealand,9.387647,11.630554,8.681386,14.841020,15.890367,14.612397
188,Finland,8.822673,12.008534,13.334918,11.331042,10.266815,13.934003
189,Ireland,7.067697,0.880739,12.725540,17.034735,11.528193,13.482017
190,Denmark,7.007698,12.078120,13.298125,13.198061,2.342698,12.566491
